In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/pasha00786/AmEx/master/data.csv')
data.head()

,Unnamed: 0,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,0,230032.0,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,...,1.0,73.780,82.5470,0.08696,10.0,0.63899,0.0,0.0,0.0,0.0
1,1,230033.0,1846.0,0.8095,0.000,0.0,0.0,102.0,7532.0,3171.0,...,0.0,99.129,78.8835,0.00000,13.0,0.63836,0.0,0.0,1.0,1.0
2,2,230034.0,1745.0,0.4001,0.000,0.0,0.0,297.0,2536.0,2478.0,...,0.0,100.000,29.2900,0.00000,1.0,1.00000,0.0,0.0,0.0,1.0
3,3,230035.0,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,...,0.0,96.272,78.8835,0.15385,3.0,0.53241,0.0,0.0,1.0,0.0
4,4,230036.0,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,...,0.0,115.019,78.8835,0.00000,1.0,0.92665,0.0,0.0,1.0,0.0


In [0]:
data.drop('Unnamed: 0', axis = 1, inplace = True)

In [0]:
X = data.drop(['application_key', 'default_ind'], axis = 1)
y = data['default_ind']

In [0]:
data_dmatrix = xgb.DMatrix(data = X, label =y)

In [0]:
clf = xgb.XGBClassifier()

In [0]:
param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

In [0]:
randomized_clf = RandomizedSearchCV(estimator=clf, param_distributions= param_grid, scoring = 'roc_auc', cv = 2, n_iter = 5, verbose = 1)

In [22]:
randomized_clf.fit(X,y)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.3min finished


RandomizedSearchCV(cv=2, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'silent': [False], 'max_depth': [6, 10, 15, 20], 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0, 3], 'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0], 'gamma': [0, 0.25, 0.5, 1.0], 'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0], 'n_estimators': [100]},
          pre_dispatch='2*n_jobs', random_s

In [23]:
best_score = randomized_clf.best_score_
best_params = randomized_clf.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

Best score: 0.8275207395113745
Best params: 
colsample_bylevel: 0.6
colsample_bytree: 1.0
gamma: 0.25
learning_rate: 0.2
max_depth: 20
min_child_weight: 0.5
n_estimators: 100
reg_lambda: 50.0
silent: False
subsample: 0.9


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [0]:
bestXGbFitted = randomized_clf.best_estimator_.fit(X_train, y_train)

In [29]:
test_labels=bestXGbFitted.predict_proba(X_test)[:,1]
roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

0.8268275922243876

In [30]:
bestXGbFitted.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8054

In [0]:
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)
X_train_1 = np.array(X_train)
X_train_transform = transformer.transform(X_train_1)
X_test_1 = np.array(X_test)
X_test_transform = transformer.transform(X_test_1)

In [0]:
bestXGbFitted_transformed = randomized_clf.best_estimator_.fit(X_train_transform, y_train)

In [60]:
test_labels=bestXGbFitted.predict_proba(X_test_transform)[:,1]
roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

0.8271959155003624

In [0]:
#Outliers Removal

In [0]:
dataset = pd.read_csv('https://raw.githubusercontent.com/pasha00786/AmEx/master/data.csv')
dataset.drop(['Unnamed: 0', 'application_key'], axis = 1, inplace = True)

In [39]:
dataset.shape

(80000, 48)

In [40]:
Q1 = dataset.quantile(0.25)
Q3 = dataset.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

dataset_out = dataset[~((dataset < (Q1 - 1.5 * IQR)) |(dataset > (Q3 + 1.5 * IQR))).any(axis=1)]

mvar1            123.000000
mvar2              1.198300
mvar3              6.373000
mvar4              0.000000
mvar5              0.000000
mvar6            698.000000
mvar7          14247.250000
mvar8           5353.000000
mvar9          31594.500000
mvar10         32605.250000
mvar11             0.000000
mvar12          5861.000000
mvar13         11545.500000
mvar14         81262.000000
mvar15         71868.250000
mvar16             0.000000
mvar17             0.000000
mvar18             0.000000
mvar19             1.000000
mvar20             1.000000
mvar21            36.912500
mvar22          2996.335000
mvar23             0.000000
mvar24            47.255000
mvar25          1339.000000
mvar26          2950.000000
mvar27          2586.000000
mvar28         30356.000000
mvar29          3954.000000
mvar30           426.000000
mvar31             0.000000
mvar32           502.000000
mvar33             9.250000
mvar34             0.000000
mvar35             0.000000
mvar36             5

In [41]:
dataset_out.shape

(1470, 48)

In [0]:
X_out = dataset_out.drop('default_ind', axis = 1)
y_out = dataset_out['default_ind']

In [0]:
X_out_train, X_out_test, y_out_train, y_out_test = train_test_split(X_out, y_out, test_size=0.25)

In [0]:
bestXGbFitted = randomized_clf.best_estimator_.fit(X_out_train, y_out_train)

In [55]:
bestXGbFitted.score(X_out_test, y_out_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


1.0

In [0]:
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, nfold=3, num_boost_round=5, metrics="error", as_pandas=True, seed=123)

In [56]:
X_out.head()

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,...,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
32,1837.0,0.0234,0.000,0.0,0.0,297.0,700.0,2478.0,8275.0,6745.0,...,0.0,0.0,100.0,78.8835,0.000,5.0,1.00000,0.0,0.0,1.0
125,1790.0,1.5634,0.413,0.0,0.0,277.0,14224.0,1634.0,4311.0,16135.0,...,2.0,0.0,100.0,78.8835,0.125,3.0,0.87078,0.0,0.0,0.0
140,1888.0,0.1126,0.000,0.0,0.0,1373.0,22478.0,1373.0,38885.0,48431.0,...,0.0,0.0,100.0,78.8835,0.000,9.0,0.50591,0.0,0.0,1.0
210,1784.0,0.6106,0.000,0.0,0.0,1765.0,1982.0,1982.0,12895.0,3747.0,...,0.0,0.0,100.0,78.8835,0.250,0.0,1.00000,0.0,0.0,1.0
223,1742.0,0.0386,0.300,0.0,0.0,297.0,6967.0,2478.0,14171.0,9492.5,...,3.0,0.0,100.0,78.8835,0.200,5.0,0.73703,0.0,0.0,0.0


In [57]:
y_out.head()

32     0.0
125    0.0
140    0.0
210    0.0
223    0.0
Name: default_ind, dtype: float64